Group members:
1. Rashmi Shah - 301846572
2. Chaitali Patil - 301877356

In [1]:
import numpy as np

In [2]:
# Sigmoid activation function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [3]:
# Derivative of sigmoid function
def sigmoid_prime(x):
    return sigmoid(x) * (1 - sigmoid(x))

In [4]:
# Initializing a neural network
# Input_size= no. of nodes in the input layer
# Hidden_size =  no. of nodes in the hidden layer
# Output_size = no. of nodes in the output layer
def initialize_network(input_size, hidden_size, output_size):         # Initializing the weights of a neural network with random values
    network = {                                                       # Initializing a dictionary to hold the network weights
        'weights_input_hidden': np.random.rand(hidden_size, input_size) * 0.02 - 0.01,
        'weights_hidden_output': np.random.rand(output_size, hidden_size) * 0.02 - 0.01
    }
    return network

In [5]:
# Forward propagate an input to calculate the output of a neural network
def forward_propagate(network, input_data):
    hidden_output = sigmoid(np.dot(network['weights_input_hidden'], input_data))
    output = sigmoid(np.dot(network['weights_hidden_output'], hidden_output))
    return output

In [6]:
# Performing backpropagation to update the weights of a neural network
def backpropagate(network, input_data, target_output, learning_rate):

    # Forward pass: calculating the output of the hidden layer and the final output
    hidden_output = sigmoid(np.dot(network['weights_input_hidden'], input_data))
    output = sigmoid(np.dot(network['weights_hidden_output'], hidden_output))

    # Calculating the errors for the output layer and the hidden layer
    output_error = target_output - output            # Error in the output layer
    hidden_error = np.dot(network['weights_hidden_output'].T, output_error)

    # Calculating the deltas for the output layer and the hidden layer
    delta_output = output_error * sigmoid_prime(output)         # Delta for the output layer
    delta_hidden = hidden_error * sigmoid_prime(hidden_output)          # Delta for the hidden layer

    # Updating the weights
    network['weights_hidden_output'] += learning_rate * np.outer(delta_output, hidden_output)
    network['weights_input_hidden'] += learning_rate * np.outer(delta_hidden, input_data)


In [7]:
# Training the network with online training
def train_network_online(network, input_data, target_output, learning_rate, epochs):
    errors = []                 # Storing errors for each epoch in lists
    accuracies = []             # Storing accuracies for each epoch in lists

    # Looping through each epoch
    for epoch in range(epochs):
        total_squared_error = 0              # Initializing total squared error for the epoch
        correct_predictions = 0              # Initializing number of correct predictions for the epoch

        # Looping through each data point in the input data
        for i in range(len(input_data)):
            backpropagate(network, input_data[i], target_output[i], learning_rate)       # Performing backpropagation to update weights
            prediction = forward_propagate(network, input_data[i])[0]           # Forward propagation to make predictions
            total_squared_error += (target_output[i] - prediction) ** 2          # Calculating total squared error

            # Checking for correct predictions
            if round(prediction) == target_output[i]:
                correct_predictions += 1
        errors.append(total_squared_error)                             # Calculating and storing total squared error and accuracy for the epoch
        accuracies.append(correct_predictions / len(input_data))
        print(f"Epoch {epoch+1}: Error = {total_squared_error}, Accuracy = {accuracies[-1]}")
    return errors, accuracies                                          # Returning lists of errors and accuracies for all epochs

In [8]:
# Training the network with batch training
def train_network_batch(network, input_data, target_output, learning_rate, epochs):

    # Storing errors and accuracies for each epoch in lists
    errors = []
    accuracies = []

     # Looping through each epoch
    for epoch in range(epochs):
        total_squared_error = 0      # Total squared error for the epoch
        correct_predictions = 0      # Number of correct predictions for the epoch

        # Initializing delta arrays for weights update
        delta_output_batch = np.zeros_like(network['weights_hidden_output'])
        delta_hidden_batch = np.zeros_like(network['weights_input_hidden'])

        # Looping through each data point in the input data
        for i in range(len(input_data)):

            # Forward pass
            hidden_output = sigmoid(np.dot(network['weights_input_hidden'], input_data[i]))
            output = sigmoid(np.dot(network['weights_hidden_output'], hidden_output))

            # Calculating Error
            output_error = target_output[i] - output
            hidden_error = np.dot(network['weights_hidden_output'].T, output_error)

            # Backpropagation - Calculating deltas
            delta_output = output_error * sigmoid_prime(output)
            delta_hidden = hidden_error * sigmoid_prime(hidden_output)

            # Accumulating deltas for batch update
            delta_output_batch += np.outer(delta_output, hidden_output)
            delta_hidden_batch += np.outer(delta_hidden, input_data[i])

            # Calculating total squared error
            prediction = output[0]
            total_squared_error += (target_output[i] - prediction) ** 2

            # Checking for correct predictions
            if round(prediction) == target_output[i]:
                correct_predictions += 1

        # Updating weights using averaged deltas
        network['weights_hidden_output'] += learning_rate * delta_output_batch / len(input_data)
        network['weights_input_hidden'] += learning_rate * delta_hidden_batch / len(input_data)

        # Calculating and storing mean squared error and accuracy for the epoch
        errors.append(total_squared_error)
        accuracies.append(correct_predictions / len(input_data))

        print(f"Epoch {epoch+1}: Error = {total_squared_error}, Accuracy = {accuracies[-1]}")     # Printing epoch information
    return errors, accuracies           # Returning lists of errors and accuracies for all epochs

In [9]:
# Providing data for training
input_data = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
target_output = np.array([[0], [1], [1], [0]])


In [10]:
# Learning Experiment with Online Training : 5 times results for the experiment
print("Learning Experiment with Online Training:")

# Looping through 5 experiments
for i in range(5):
    print(f"\nExperiment {i+1}:")           # Printing experiment number
    network = initialize_network(input_size=2, hidden_size=4, output_size=1)            # Initializing a neural network for the experiment

    # Printing initial random weights
    print("Initial Random Weights:")
    print("weights_input_hidden:", network['weights_input_hidden'])
    print("weights_hidden_output:", network['weights_hidden_output'])

    # Training the network using online training
    errors, accuracies = train_network_online(network, input_data, target_output, learning_rate=0.1, epochs=20)

    # Printing final weights after training
    print("Final Weights:")
    print("weights_input_hidden:", network['weights_input_hidden'])
    print("weights_hidden_output:", network['weights_hidden_output'])

    # Printing final weights after training
    print("Squared Error at the end of training:", errors[-1])
    print("Accuracy at the end of training:", accuracies[-1])

Learning Experiment with Online Training:

Experiment 1:
Initial Random Weights:
weights_input_hidden: [[ 0.00895718  0.00665834]
 [ 0.00991506  0.006829  ]
 [ 0.00872468 -0.00059321]
 [ 0.00171211  0.0049221 ]]
weights_hidden_output: [[ 0.00350874 -0.0030987   0.00056621  0.00789257]]
Epoch 1: Error = [0.99407907], Accuracy = 0.75
Epoch 2: Error = [0.99407948], Accuracy = 0.75
Epoch 3: Error = [0.99407989], Accuracy = 0.75
Epoch 4: Error = [0.99408031], Accuracy = 0.75
Epoch 5: Error = [0.99408074], Accuracy = 0.75
Epoch 6: Error = [0.99408118], Accuracy = 0.75
Epoch 7: Error = [0.99408162], Accuracy = 0.75
Epoch 8: Error = [0.99408207], Accuracy = 0.75
Epoch 9: Error = [0.99408253], Accuracy = 0.75
Epoch 10: Error = [0.994083], Accuracy = 0.75
Epoch 11: Error = [0.99408347], Accuracy = 0.75
Epoch 12: Error = [0.99408394], Accuracy = 0.75
Epoch 13: Error = [0.99408443], Accuracy = 0.75
Epoch 14: Error = [0.99408492], Accuracy = 0.75
Epoch 15: Error = [0.99408541], Accuracy = 0.75
Epoc

In [11]:
# Learning Experiment with Batch Training: 5 times results for the experiment
print("\nLearning Experiment with Batch Training:")

# Looping through 5 experiments
for i in range(5):
    print(f"\nExperiment {i+1}:")                 # Printing experiment number

    network = initialize_network(input_size=2, hidden_size=4, output_size=1)               # Initializing a neural network for the experiment

     # Printing initial random weights
    print("Initial Random Weights:")
    print("weights_input_hidden:", network['weights_input_hidden'])
    print("weights_hidden_output:", network['weights_hidden_output'])

    # Training the network using batch training
    errors, accuracies = train_network_batch(network, input_data, target_output, learning_rate=0.1, epochs=20)

    # Printing final weights after training
    print("Final Weights:")
    print("weights_input_hidden:", network['weights_input_hidden'])
    print("weights_hidden_output:", network['weights_hidden_output'])

    # Printing squared error and accuracy at the end of training
    print("Squared Error at the end of training:", errors[-1])
    print("Accuracy at the end of training:", accuracies[-1])


Learning Experiment with Batch Training:

Experiment 1:
Initial Random Weights:
weights_input_hidden: [[-0.00692839  0.00825033]
 [ 0.00272404  0.00958171]
 [-0.0002865  -0.00129647]
 [ 0.0045393  -0.00067637]]
weights_hidden_output: [[ 0.00184069 -0.00350716  0.00148668 -0.00928286]]
Epoch 1: Error = [1.00000562], Accuracy = 0.5
Epoch 2: Error = [1.00000555], Accuracy = 0.5
Epoch 3: Error = [1.00000549], Accuracy = 0.5
Epoch 4: Error = [1.00000542], Accuracy = 0.5
Epoch 5: Error = [1.00000536], Accuracy = 0.5
Epoch 6: Error = [1.0000053], Accuracy = 0.5
Epoch 7: Error = [1.00000524], Accuracy = 0.5
Epoch 8: Error = [1.00000517], Accuracy = 0.5
Epoch 9: Error = [1.00000511], Accuracy = 0.5
Epoch 10: Error = [1.00000505], Accuracy = 0.5
Epoch 11: Error = [1.00000499], Accuracy = 0.5
Epoch 12: Error = [1.00000494], Accuracy = 0.5
Epoch 13: Error = [1.00000488], Accuracy = 0.5
Epoch 14: Error = [1.00000482], Accuracy = 0.5
Epoch 15: Error = [1.00000476], Accuracy = 0.5
Epoch 16: Error = 